In [1]:
from pyspark.sql import SparkSession, SQLContext, GroupedData, HiveContext
from pyspark.sql.functions import *
from pyspark.sql.functions import date_add as d_add
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import year, month, dayofmonth, weekofyear, date_format
from pyspark.sql import functions as F
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import lit
from pyspark.sql import Row
import os 
import pandas as pd 
import numpy as np
# Increased memory
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.config("spark.python.worker.memory", "15g") \
.enableHiveSupport().getOrCreate()
print("Setup complete")

Setup complete


In [2]:
os.listdir(os.getcwd()+'/data')

['I94_SAS_Labels_Descriptions.sas',
 'airport-codes_csv.csv',
 'arrival_port_map.csv',
 'country_details.csv',
 'flight_number.csv',
 'immigration_data_sample.csv',
 'mismatch.csv',
 'sas_data',
 'us-cities-demographics.csv']

In [3]:
# Setting up Files to read
data_dir = os.path.join(os.getcwd(), 'data')
country_detail_path = os.path.join(data_dir, r'country_details.csv')
airport_mismatch_path =  os.path.join(data_dir,r'mismatch.csv') 
airport_data = os.path.join(data_dir, 'airport-codes_csv.csv')
arrival_port_map = os.path.join(data_dir, 'arrival_port_map.csv')
demographic_data = os.path.join(data_dir, 'us-cities-demographics.csv')
sas_path = os.path.join(data_dir, 'sas_data')
sas_path_specific = os.path.join(sas_path, 'part-00001-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet')
output_path = os.path.join(os.getcwd(), 'spark_output_parquet')

In [4]:
# util functions
def shape(d):
    print((d.count(), len(d.columns)))
    
def detail(d):
    print("Shape ==>", (d.count(), len(d.columns)))
    print("Schema ::")
    d.printSchema()
    d.show(2)

In [5]:
!ls /home/jovyan/work/data/sas_data

part-00000-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet
part-00001-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet
part-00002-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet
part-00003-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet
part-00004-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet
part-00005-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet
part-00006-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet
part-00007-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet
part-00008-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet
part-00009-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet
part-00010-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet
part-00011-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet
part-00012-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet
part-00013-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet
_SUCCESS


In [6]:
# Reading immigration data
print(sas_path)
df=spark.read.parquet(sas_path_specific)
detail(df)

/home/jovyan/work/data/sas_data
Shape ==> (220160, 28)
Schema ::
root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- a

In [7]:
import json
def json_beautifier(d):
    s = json.loads(d.schema.json())['fields']
    cols = ['Field', 'Type', 'Nullable']
    schema_df = pd.DataFrame(columns=cols, data={
        'Field': [x['name'] for x in s],
        'Type': [x['type'] for x in s],
        'Nullable': [x['nullable'] for x in s]
    })
    return schema_df

x = json_beautifier(df)
x.to_csv('abc.csv')


In [8]:
def write_to_parquet(table, file_name):
    """
    Write the table as parquet file
    Parameters:
        table
        Name of the file
    Returns:
        Outputs the table as parquet file to a folder
    """
    file_output = os.path.join(output_path, file_name)
    table.write.mode("overwrite").parquet(file_output)

In [9]:
print("Immigration_table schema:")
df.printSchema()

Immigration_table schema:
root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- a

In [10]:
# sampling down to run with a smaller dataset
df_small = df.sample(withReplacement=False, fraction=0.4, seed=3)
shape(df_small)
shape(df)

(87680, 28)
(220160, 28)


In [11]:
# Adjusting the immigration dataframe
df2 = df_small \
.withColumn("cicid", col("cicid").cast("integer")) \
.withColumn("year", col("i94yr").cast("integer")) \
.drop("i94yr") \
.withColumn("month", col("i94mon").cast("integer")) \
.drop("i94mon") \
.withColumn("bornCountry", col("i94cit").cast("integer")) \
.drop("i94cit") \
.withColumn("residentCountry", col("i94res").cast("integer")) \
.drop("i94res") \
.withColumnRenamed("i94port", "arrivalPort") \
.withColumn("mode", col("i94mode").cast("integer")) \
.drop("i94mode") \
.withColumnRenamed("i94addr", "arrivalAddress") \
.withColumn("age", col("i94bir").cast("integer")) \
.drop("i94bir") \
.withColumn("visaId", col("i94visa").cast("integer")) \
.drop("i94visa") \
.withColumnRenamed("entdepa", "arrivalFlag") \
.withColumnRenamed("entdepd", "departureFlag") \
.withColumnRenamed("entdepu", "updateFlag") \
.withColumnRenamed("matflag", "matchFlag") \
.withColumn("birthYear", col("biryear").cast("integer")) \
.drop("biryear") \
.withColumnRenamed("fltno", "flightNumber") \
.withColumnRenamed("visaType", "visaType") \
.withColumn("sasDate", to_date(lit("01/01/1960"), "MM/dd/yyyy")) \
.withColumn("arrdate", col("arrdate").cast("integer")) \
.withColumn("depdate", col("depdate").cast("integer")) \
.withColumn("arrivalDate", expr("date_add(sasDate, arrdate)")) \
.withColumn("departureDate", expr("date_add(sasDate, depdate)")) \
.drop("sasDate", "arrdate", "depdate", "count", "admnum", "dtadfile", "visapost", "occup", "dtaddto", "insnum")



In [12]:
print("New Immigration data schema:")
df2.printSchema()

New Immigration data schema:
root
 |-- cicid: integer (nullable = true)
 |-- arrivalPort: string (nullable = true)
 |-- arrivalAddress: string (nullable = true)
 |-- arrivalFlag: string (nullable = true)
 |-- departureFlag: string (nullable = true)
 |-- updateFlag: string (nullable = true)
 |-- matchFlag: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- flightNumber: string (nullable = true)
 |-- visaType: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- bornCountry: integer (nullable = true)
 |-- residentCountry: integer (nullable = true)
 |-- mode: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- visaId: integer (nullable = true)
 |-- birthYear: integer (nullable = true)
 |-- arrivalDate: date (nullable = true)
 |-- departureDate: date (nullable = true)



In [13]:
# Time table creation
time_table = df2.select(['arrivalDate'])\
                    .withColumnRenamed('arrivalDate','time') 

time_table = time_table \
             .withColumn('day', F.dayofmonth('time')) \
             .withColumn('month', F.month('time')) \
             .withColumn('year', F.year('time')) \
             .withColumn('week', F.weekofyear('time')) \
             .withColumn('weekday', F.dayofweek('time'))\
             .dropDuplicates()
write_to_parquet(time_table,"time_table.parquet" )

In [14]:
detail(time_table)

Shape ==> (4, 6)
Schema ::
root
 |-- time: date (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- weekday: integer (nullable = true)

+----------+---+-----+----+----+-------+
|      time|day|month|year|week|weekday|
+----------+---+-----+----+----+-------+
|2016-04-05|  5|    4|2016|  14|      3|
|2016-04-02|  2|    4|2016|  13|      7|
+----------+---+-----+----+----+-------+
only showing top 2 rows



In [15]:
# Person table creation
person_table = df2.select(['birthYear','gender']) \
                 .dropDuplicates() \
                 .withColumn("personId", \
                        monotonically_increasing_id())\
                 .withColumnRenamed("gender", "person_gender")
write_to_parquet(person_table,"person_table.parquet" )

In [16]:
detail(person_table)

Shape ==> (286, 3)
Schema ::
root
 |-- birthYear: integer (nullable = true)
 |-- person_gender: string (nullable = true)
 |-- personId: long (nullable = false)

+---------+-------------+--------+
|birthYear|person_gender|personId|
+---------+-------------+--------+
|     1972|            M|       0|
|     1959|         null|       1|
+---------+-------------+--------+
only showing top 2 rows



In [17]:
# Country table creation
country_table = df2.select('bornCountry').distinct().withColumnRenamed("bornCountry", "countryId") \
            
country_name = spark.read.csv(country_detail_path).dropna()
country_name = country_name.withColumn("_c0", col("_c0").cast("integer"))\
.withColumnRenamed("_c0", "countryId_").withColumnRenamed("_c1", "countryName")
                
country_table = country_table.join(country_name, (country_table.countryId == country_name.countryId_)).drop('countryId_')
write_to_parquet(country_table,"country_table.parquet" )

In [18]:
detail(country_table)

Shape ==> (173, 2)
Schema ::
root
 |-- countryId: integer (nullable = true)
 |-- countryName: string (nullable = true)

+---------+-------------------+
|countryId|        countryName|
+---------+-------------------+
|      392|               MALI|
|      516|TRINIDAD AND TOBAGO|
+---------+-------------------+
only showing top 2 rows



In [19]:
# Arrival table creation
arrival_port_table = df2.groupby('arrivalPort') \
                     .agg(F.countDistinct("arrivalAddress"))\
                     .withColumnRenamed("count(arrivalAddress)","numArrivalAddress") \
                     .dropna() \
                     .dropDuplicates()\
                     .withColumn("arrivalPortId", \
                        monotonically_increasing_id())
arrival_data_map = spark.read.csv(arrival_port_map).dropna()\
.withColumnRenamed("_c0","arrivalPort_") \
.withColumnRenamed("_c1","stateCode")

arrival_port_table = arrival_port_table\
.join(arrival_data_map,  (arrival_port_table.arrivalPort == arrival_data_map.arrivalPort_))\

write_to_parquet(arrival_port_table,"arrival_port_table.parquet")

In [20]:
detail(arrival_port_table)

Shape ==> (98, 5)
Schema ::
root
 |-- arrivalPort: string (nullable = true)
 |-- numArrivalAddress: long (nullable = false)
 |-- arrivalPortId: long (nullable = false)
 |-- arrivalPort_: string (nullable = true)
 |-- stateCode: string (nullable = true)

+-----------+-----------------+-------------+------------+---------+
|arrivalPort|numArrivalAddress|arrivalPortId|arrivalPort_|stateCode|
+-----------+-----------------+-------------+------------+---------+
|        FMY|               33|            0|         FMY|       FL|
|        BGM|                2|            1|         BGM|       ME|
+-----------+-----------------+-------------+------------+---------+
only showing top 2 rows



In [21]:
# Status table creation
status_table = df2.select(['arrivalFlag','departureFlag', 'updateFlag','matchFlag']) \
               .dropna() \
               .dropDuplicates() \
               .withColumn("statusFlagId", monotonically_increasing_id()) 
write_to_parquet(status_table,"status_table.parquet")

In [22]:
detail(status_table)

Shape ==> (4, 5)
Schema ::
root
 |-- arrivalFlag: string (nullable = true)
 |-- departureFlag: string (nullable = true)
 |-- updateFlag: string (nullable = true)
 |-- matchFlag: string (nullable = true)
 |-- statusFlagId: long (nullable = false)

+-----------+-------------+----------+---------+------------+
|arrivalFlag|departureFlag|updateFlag|matchFlag|statusFlagId|
+-----------+-------------+----------+---------+------------+
|          T|            O|         U|        M|           0|
|          G|            N|         U|        M|           1|
+-----------+-------------+----------+---------+------------+
only showing top 2 rows



In [23]:
# Creating airport table
airport = spark.read.format("csv").option("header", "true").option("delimiter", ",").load(airport_data)
y = json_beautifier(airport)
y.to_csv('airport_schema.csv')
detail(airport)
airport = airport.where(\
            (col("iso_country") == "US")\
          & (col("iata_code").isNotNull())\
          & (col("type").isin("large_airport", "medium_airport", "small_airport"))) \
            .withColumn("isoRegion", substring(col("iso_region"), 4, 2)) \
            .dropDuplicates()

airport = airport\
.withColumn("lat", split(col("coordinates"), ",").getItem(0))\
.withColumn("long", split(col("coordinates"), ",").getItem(1))

airport = airport\
.withColumn("lat", col("lat").cast("float"))\
.withColumn("long", col("long").cast("float"))\
.drop('local_code', 'elevation_ft', 'continent', 'coordinates')

airport_table = airport.select(['ident', 'type', 'name', 'isoRegion', 'municipality',
                                'gps_code', 'iata_code','iso_country', 'lat', 'long']) \
               .dropDuplicates().dropna()
write_to_parquet(airport_table,"airport_table.parquet" )



Shape ==> (55075, 12)
Schema ::
root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)

+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|  00A|     he

In [24]:
detail(airport)

Shape ==> (1865, 11)
Schema ::
root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- isoRegion: string (nullable = true)
 |-- lat: float (nullable = true)
 |-- long: float (nullable = true)

+-----+--------------+--------------------+-----------+----------+--------------------+--------+---------+---------+--------+-------+
|ident|          type|                name|iso_country|iso_region|        municipality|gps_code|iata_code|isoRegion|     lat|   long|
+-----+--------------+--------------------+-----------+----------+--------------------+--------+---------+---------+--------+-------+
| KBPT|medium_airport|Southeast Texas R...|         US|     US-TX|Beaumont/Port Arthur|    KBPT|      BPT|       TX|-94.0207|29.9508

In [25]:
# Creating demographics table
dem = spark.read.format("csv").option("header", "true").option("delimiter", ";").load(demographic_data)
z= json_beautifier(dem)
z.to_csv('dem.csv')
detail(dem)
dem_table = dem \
.groupBy(col("State Code").alias("stateCode"), col("State").alias("state")).agg(
        round(mean('Median Age'), 2).alias("medianAge"),\
        sum("Total Population").alias("totalPopulation"),\
        sum("Male Population").alias("malePopulation"), \
        sum("Female Population").alias("femalePopulation"),\
        sum("Number of Veterans").alias("numberOfVeterans"),\
        sum("Foreign-born").alias("foreignBorn"), \
        round(mean("Average Household Size"),2).alias("averageHouseholdSize")
).dropna()
write_to_parquet(dem_table,"dem_table.parquet" )


Shape ==> (2891, 12)
Schema ::
root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: string (nullable = true)
 |-- Male Population: string (nullable = true)
 |-- Female Population: string (nullable = true)
 |-- Total Population: string (nullable = true)
 |-- Number of Veterans: string (nullable = true)
 |-- Foreign-born: string (nullable = true)
 |-- Average Household Size: string (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: string (nullable = true)

+-------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+------------------+-----+
|         City|        State|Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|Average Household Size|State Code|              Race|Count|
+-------------+-------------+----------+---------------+-----------------+-

In [26]:
detail(dem)

Shape ==> (2891, 12)
Schema ::
root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: string (nullable = true)
 |-- Male Population: string (nullable = true)
 |-- Female Population: string (nullable = true)
 |-- Total Population: string (nullable = true)
 |-- Number of Veterans: string (nullable = true)
 |-- Foreign-born: string (nullable = true)
 |-- Average Household Size: string (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: string (nullable = true)

+-------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+------------------+-----+
|         City|        State|Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|Average Household Size|State Code|              Race|Count|
+-------------+-------------+----------+---------------+-----------------+-

In [27]:
# Merging different tables
small_table = df2.select(['cicid', 'arrivalDate','departureDate','mode','bornCountry', 
                          'visaId','visaType',
                         'gender','arrivalPort', 'birthYear','matchFlag']) \
                .dropna() \
                .dropDuplicates(['cicid'])


In [28]:
# merge with time, commenting the code to have arrivalDate as the column
# small_table = small_table.withColumnRenamed("arrivalDate", "time")  
detail(small_table)

Shape ==> (71138, 11)
Schema ::
root
 |-- cicid: integer (nullable = true)
 |-- arrivalDate: date (nullable = true)
 |-- departureDate: date (nullable = true)
 |-- mode: integer (nullable = true)
 |-- bornCountry: integer (nullable = true)
 |-- visaId: integer (nullable = true)
 |-- visaType: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- arrivalPort: string (nullable = true)
 |-- birthYear: integer (nullable = true)
 |-- matchFlag: string (nullable = true)

+------+-----------+-------------+----+-----------+------+--------+------+-----------+---------+---------+
| cicid|arrivalDate|departureDate|mode|bornCountry|visaId|visaType|gender|arrivalPort|birthYear|matchFlag|
+------+-----------+-------------+----+-----------+------+--------+------+-----------+---------+---------+
|459674| 2016-04-03|   2016-04-07|   1|        135|     2|      WT|     M|        ATL|     1955|        M|
|459682| 2016-04-03|   2016-04-07|   1|        135|     1|      WB|     M|        ATL|  

In [29]:
# merge with status
small_table = small_table.join(status_table.select(['statusFlagId', 'matchFlag']), (small_table.matchFlag == status_table.matchFlag) , how = 'inner') \
                       .drop('matchFlag') \
                       .dropna() \
                       .dropDuplicates() 
detail(small_table)

Shape ==> (284552, 11)
Schema ::
root
 |-- cicid: integer (nullable = true)
 |-- arrivalDate: date (nullable = true)
 |-- departureDate: date (nullable = true)
 |-- mode: integer (nullable = true)
 |-- bornCountry: integer (nullable = true)
 |-- visaId: integer (nullable = true)
 |-- visaType: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- arrivalPort: string (nullable = true)
 |-- birthYear: integer (nullable = true)
 |-- statusFlagId: long (nullable = false)

+------+-----------+-------------+----+-----------+------+--------+------+-----------+---------+------------+
| cicid|arrivalDate|departureDate|mode|bornCountry|visaId|visaType|gender|arrivalPort|birthYear|statusFlagId|
+------+-----------+-------------+----+-----------+------+--------+------+-----------+---------+------------+
|459674| 2016-04-03|   2016-04-07|   1|        135|     2|      WT|     M|        ATL|     1955|           3|
|459674| 2016-04-03|   2016-04-07|   1|        135|     2|      WT|     M

In [30]:

# merge with country
small_table = small_table.join(country_table, (small_table.bornCountry == country_table.countryId) , how = 'inner') \
                         .dropna() \
                        .drop('bornCountry') \
                       .dropDuplicates()
detail(small_table)

Shape ==> (249188, 12)
Schema ::
root
 |-- cicid: integer (nullable = true)
 |-- arrivalDate: date (nullable = true)
 |-- departureDate: date (nullable = true)
 |-- mode: integer (nullable = true)
 |-- visaId: integer (nullable = true)
 |-- visaType: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- arrivalPort: string (nullable = true)
 |-- birthYear: integer (nullable = true)
 |-- statusFlagId: long (nullable = false)
 |-- countryId: integer (nullable = true)
 |-- countryName: string (nullable = true)

+------+-----------+-------------+----+------+--------+------+-----------+---------+------------+---------+-----------+
| cicid|arrivalDate|departureDate|mode|visaId|visaType|gender|arrivalPort|birthYear|statusFlagId|countryId|countryName|
+------+-----------+-------------+----+------+--------+------+-----------+---------+------------+---------+-----------+
|626141| 2016-04-04|   2016-04-10|   1|     2|      B2|     M|        NYC|     2010|           3|      101|    A

In [31]:

# merge with person
small_table = small_table.join(person_table.select(['personId', 'person_gender', 'birthYear']),
                               ((small_table.gender == person_table.person_gender) & \
                               (small_table.birthYear == person_table.birthYear)),\
                               how = 'inner') \
                      .drop('person_gender','birthYear') \
                      .dropna() \
                      .dropDuplicates()
detail(small_table)

Shape ==> (249188, 12)
Schema ::
root
 |-- cicid: integer (nullable = true)
 |-- arrivalDate: date (nullable = true)
 |-- departureDate: date (nullable = true)
 |-- mode: integer (nullable = true)
 |-- visaId: integer (nullable = true)
 |-- visaType: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- arrivalPort: string (nullable = true)
 |-- statusFlagId: long (nullable = false)
 |-- countryId: integer (nullable = true)
 |-- countryName: string (nullable = true)
 |-- personId: long (nullable = false)

+------+-----------+-------------+----+------+--------+------+-----------+------------+---------+-----------+--------+
| cicid|arrivalDate|departureDate|mode|visaId|visaType|gender|arrivalPort|statusFlagId|countryId|countryName|personId|
+------+-----------+-------------+----+------+--------+------+-----------+------------+---------+-----------+--------+
|626141| 2016-04-04|   2016-04-10|   1|     2|      B2|     M|        NYC|           3|      101|    ALBANIA|     104|

In [32]:
detail(small_table)

Shape ==> (249188, 12)
Schema ::
root
 |-- cicid: integer (nullable = true)
 |-- arrivalDate: date (nullable = true)
 |-- departureDate: date (nullable = true)
 |-- mode: integer (nullable = true)
 |-- visaId: integer (nullable = true)
 |-- visaType: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- arrivalPort: string (nullable = true)
 |-- statusFlagId: long (nullable = false)
 |-- countryId: integer (nullable = true)
 |-- countryName: string (nullable = true)
 |-- personId: long (nullable = false)

+------+-----------+-------------+----+------+--------+------+-----------+------------+---------+-----------+--------+
| cicid|arrivalDate|departureDate|mode|visaId|visaType|gender|arrivalPort|statusFlagId|countryId|countryName|personId|
+------+-----------+-------------+----+------+--------+------+-----------+------------+---------+-----------+--------+
|626141| 2016-04-04|   2016-04-10|   1|     2|      B2|     M|        NYC|           3|      101|    ALBANIA|     104|

In [33]:
# merge with arrival
small_table = small_table.join(arrival_port_table.select(['arrivalPortId', 'arrivalPort_', 'stateCode']),  \
                              (small_table.arrivalPort == arrival_port_table.arrivalPort_) , how = 'inner') \
                       .drop('arrivalPort_')\
                       .dropna()\
                       .dropDuplicates()
small_table.show(2)
detail(small_table)

+------+-----------+-------------+----+------+--------+------+-----------+------------+---------+-----------+--------+-------------+---------+
| cicid|arrivalDate|departureDate|mode|visaId|visaType|gender|arrivalPort|statusFlagId|countryId|countryName|personId|arrivalPortId|stateCode|
+------+-----------+-------------+----+------+--------+------+-----------+------------+---------+-----------+--------+-------------+---------+
|637761| 2016-04-04|   2016-04-10|   1|     2|      WT|     F|        AGA|           3|      111|     FRANCE|     105|           10|       GU|
|637761| 2016-04-04|   2016-04-10|   1|     2|      WT|     F|        AGA|           2|      111|     FRANCE|     105|           10|       GU|
+------+-----------+-------------+----+------+--------+------+-----------+------------+---------+-----------+--------+-------------+---------+
only showing top 2 rows

Shape ==> (240564, 14)
Schema ::
root
 |-- cicid: integer (nullable = true)
 |-- arrivalDate: date (nullable = true)


In [34]:
# merge airport data
small_table = small_table.join(airport.select(['ident', 'iata_code', 'name']),\
                               (small_table.arrivalPort == airport.iata_code),\
                               how = 'inner')\
                         .drop('iata_code') \
                         .dropna() \
                         .dropDuplicates()\
                         .withColumnRenamed("name", "airportName")

# merge demographics data
small_table = small_table.join(dem.select(['State Code', 'State']),\
                               (small_table.stateCode == dem['State Code']),\
                               how = 'inner')\
                         .drop('stateCode') \
                         .dropna() \
                         .dropDuplicates()


In [35]:
detail(small_table)

Shape ==> (100612, 17)
Schema ::
root
 |-- cicid: integer (nullable = true)
 |-- arrivalDate: date (nullable = true)
 |-- departureDate: date (nullable = true)
 |-- mode: integer (nullable = true)
 |-- visaId: integer (nullable = true)
 |-- visaType: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- arrivalPort: string (nullable = true)
 |-- statusFlagId: long (nullable = false)
 |-- countryId: integer (nullable = true)
 |-- countryName: string (nullable = true)
 |-- personId: long (nullable = false)
 |-- arrivalPortId: long (nullable = false)
 |-- ident: string (nullable = true)
 |-- airportName: string (nullable = true)
 |-- State Code: string (nullable = true)
 |-- State: string (nullable = true)

+------+-----------+-------------+----+------+--------+------+-----------+------------+---------+-----------+--------+-------------+-----+--------------------+----------+-----+
| cicid|arrivalDate|departureDate|mode|visaId|visaType|gender|arrivalPort|statusFlagId|countryI

In [36]:
list_of_tables = [country_table, time_table, arrival_port_table, person_table, status_table, airport, dem, small_table]
labels = ['country', 'time','arrival', 'person', 'status', 'airport', 'dem', 'fact']

In [37]:
for table, label in zip(list_of_tables, labels):
    f = json_beautifier(table)
    f.to_csv('{}_schema.csv'.format(label))